# Custom State for Memory Management

This notebook demonstrates how to create custom agent state schemas to track additional information beyond just conversation messages.

## Key Concepts
- **Default AgentState**: Only tracks messages[]
- **Custom AgentState**: Messages + user preferences + session data + task history
- **State Persistence**: Within and between conversations
- **Memory Management**: Context-aware responses based on history

## Benefits of Custom State
- **Persistent User Preferences**: Remember settings across conversations
- **Context-Aware Responses**: Better personalization
- **Session Management**: Handle complex workflows
- **Task Tracking**: Remember what user has done before

In [ ]:
# Import required modules
from typing_extensions import Annotated
from langchain_ollama import ChatOllama
from langchain.agents import create_agent, AgentState
from langgraph.graph.message import add_messages
import tools

## Understanding Default AgentState

First, let's understand what the default AgentState provides:

In [ ]:
print("=== Custom State for Memory Management ===")

# Define custom state schema
class CustomAgentState(AgentState):
    """Extended agent state with comprehensive memory management."""
    
    # Required: conversation history (from base AgentState)
    messages: Annotated[list, add_messages]
    
    # Custom: user preferences and session data
    user_preferences: dict = {}
    session_data: dict = {}

print("✓ CustomAgentState defined with custom tracking:")
print("  - messages: Conversation history (required)")
print("  - user_preferences: User settings and preferences")
print("  - session_data: Temporary session information")

In [ ]:
print("=== Creating Agent with Custom State ===")

model = ChatOllama(model="qwen3")

# Create agent with custom state schema
agent = create_agent(
    model,
    tools=[tools.save_user_preference, tools.get_user_preference],
    state_schema=CustomAgentState  # This enables custom state tracking
)

print("✓ Agent created with custom state schema")
print("  The agent can now track user preferences and session data")
print("  State persists throughout the conversation")

## Creating Agent with Custom State

Now let's create an agent that uses our custom state schema:

In [ ]:
print("=== Testing Custom State Features ===")

# Test with custom state data
user_message = "I prefer technical explanations and detailed examples"

print(f"User input: {user_message}")
print("Additional state data:")
print("   - user_preferences: {'style': 'technical', 'verbosity': 'detailed'}")
print("   - session_data: {'session_id': 'demo123'}")

result = agent.invoke({
    "messages": user_message,
    "user_preferences": {
        "style": "technical", 
        "verbosity": "detailed",
        "examples": "code-heavy"
    },
    "session_data": {
        "session_id": "demo123",
        "start_time": "2024-01-01T10:00:00"
    }
})

print(f"Agent response: {result['messages'][-1].content}")
print(f"Preferences maintained: {result.get('user_preferences', {})}")

print("The agent can now use this state data to personalize future responses")